Inspired by the [tutorial](https://keras.io/examples/lstm_seq2seq/), I am going to implement character-level seq2seq for paraphrasing.

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
batch_size = 16  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
# num_samples = 10000  # Number of samples to train on.

In [3]:
def load_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
        return lines

In [4]:
data_path = 'opusparcus_v2/en-train-100K.txt'
lines = load_data(data_path)

In [5]:
train_lines, val_lines = train_test_split(lines, test_size=0.20, random_state=42)

In [6]:
def parse_input_and_target_from_lines(lines, calc_characters=False):
    input_texts = []
    target_texts = []
    input_characters = set()
    target_characters = set()

    for line in lines:  # lines[: min(num_samples, len(lines) - 1)]:
        splited_lines = line.split('\t')
        if len(splited_lines) < 3:
            continue

        input_text = splited_lines[1]
        target_text = splited_lines[2]


        # input_text, target_text, _ = line.split('\t')
        # We use "tab" as the "start sequence" character
        # for the targets, and "\n" as "end sequence" character.
        target_text = '\t' + target_text + '\n'
        input_texts.append(input_text)
        target_texts.append(target_text)
        if calc_characters:
            for char in input_text:
                if char not in input_characters:
                    input_characters.add(char)
            for char in target_text:
                if char not in target_characters:
                    target_characters.add(char)

    if calc_characters:
        return input_texts, target_texts, input_characters, target_characters

    return input_texts, target_texts


In [7]:
input_texts, target_texts, input_characters, target_characters = parse_input_and_target_from_lines(train_lines, True)

In [8]:
val_input_texts, val_target_texts = parse_input_and_target_from_lines(val_lines)

In [9]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters) + 1
num_decoder_tokens = len(target_characters) + 1
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [10]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 80000
Number of unique input tokens: 105
Number of unique output tokens: 143
Max sequence length for inputs: 209
Max sequence length for outputs: 210


In [11]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
input_token_index['oov'] = len(input_token_index)

target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])
target_token_index['oov'] = len(target_token_index)

As the training data is big, and creating sparse matrix cannot be load into RAM, let's create Dataset Generator to avoid out of memory error.

https://towardsdatascience.com/keras-data-generators-and-how-to-use-them-b69129ed779c

In [12]:
from dataset_generator import DataGenerator


In [13]:
dataset = DataGenerator(input_texts=input_texts, target_texts=target_texts,
                        input_token_index=input_token_index, target_token_index=target_token_index,
                        max_encoder_seq_length=max_encoder_seq_length, num_encoder_tokens=num_encoder_tokens,
                        max_decoder_seq_length=max_decoder_seq_length, num_decoder_tokens=num_decoder_tokens,
                        batch_size=batch_size)

In [14]:
val_dataset = DataGenerator(input_texts=val_input_texts, target_texts=val_target_texts,
                        input_token_index=input_token_index, target_token_index=target_token_index,
                        max_encoder_seq_length=max_encoder_seq_length, num_encoder_tokens=num_encoder_tokens,
                        max_decoder_seq_length=max_decoder_seq_length, num_decoder_tokens=num_decoder_tokens,
                        batch_size=batch_size)

In [15]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [16]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [17]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])


In [18]:
%%time 
model.fit(dataset,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=val_dataset)


Epoch 1/100
5000/5000 [==============================] - 157s 31ms/step - loss: 0.1689 - accuracy: 0.9529 - val_loss: 0.1228 - val_accuracy: 0.9640
Epoch 2/100
5000/5000 [==============================] - 157s 31ms/step - loss: 0.1122 - accuracy: 0.9672 - val_loss: 0.1053 - val_accuracy: 0.9689
Epoch 3/100
5000/5000 [==============================] - 153s 31ms/step - loss: 0.0996 - accuracy: 0.9704 - val_loss: 0.0989 - val_accuracy: 0.9706
Epoch 4/100
5000/5000 [==============================] - 155s 31ms/step - loss: 0.0938 - accuracy: 0.9719 - val_loss: 0.0951 - val_accuracy: 0.9717
Epoch 5/100
5000/5000 [==============================] - 153s 31ms/step - loss: 0.0902 - accuracy: 0.9729 - val_loss: 0.0930 - val_accuracy: 0.9722
Epoch 6/100
5000/5000 [==============================] - 154s 31ms/step - loss: 0.0875 - accuracy: 0.9735 - val_loss: 0.0914 - val_accuracy: 0.9728
Epoch 7/100
5000/5000 [==============================] - 154s 31ms/step - loss: 0.0855 - accuracy: 0.9741 - val_

5000/5000 [==============================] - 163s 33ms/step - loss: 0.0715 - accuracy: 0.9779 - val_loss: 0.0878 - val_accuracy: 0.9750
Epoch 57/100
5000/5000 [==============================] - 162s 32ms/step - loss: 0.0714 - accuracy: 0.9779 - val_loss: 0.0879 - val_accuracy: 0.9750
Epoch 58/100
5000/5000 [==============================] - 157s 31ms/step - loss: 0.0716 - accuracy: 0.9779 - val_loss: 0.0882 - val_accuracy: 0.9748
Epoch 59/100
5000/5000 [==============================] - 160s 32ms/step - loss: 0.0713 - accuracy: 0.9779 - val_loss: 0.0877 - val_accuracy: 0.9750
Epoch 60/100
5000/5000 [==============================] - 161s 32ms/step - loss: 0.0711 - accuracy: 0.9780 - val_loss: 0.0880 - val_accuracy: 0.9750
Epoch 61/100
5000/5000 [==============================] - 161s 32ms/step - loss: 0.0711 - accuracy: 0.9780 - val_loss: 0.0880 - val_accuracy: 0.9750
Epoch 62/100
5000/5000 [==============================] - 161s 32ms/step - loss: 0.0713 - accuracy: 0.9779 - val_loss: 

In [19]:
# Save model
model.save('s2s_paraphrase.h5')

### Run several examples to see the results

In [20]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [21]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [44]:
def parse_dev_corpus(file, low_mark, high_mark):
    lines = load_data(file)
    input_texts, target_texts = [], []
    for line in lines:
        splited_lines = line.split('\t')
        if len(splited_lines) < 3:
            continue

        input_text = splited_lines[1]
        target_text = splited_lines[2]
        mark = float(splited_lines[3])
        
        if mark >= low_mark and mark <= high_mark:
#             target_text = '\t' + target_text + '\n'
            input_texts.append(input_text)
            target_texts.append(target_text)
    
    return input_texts, target_texts

    

In [45]:
dev_input_texts, dev_target_texts = parse_dev_corpus("opusparcus_v2/en-dev.txt", 3.0, 4.0)

In [46]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = dataset.create_encoder_input_item_for_text(dev_input_texts[seq_index])
    input_seq = np.array([input_seq])
#     print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    print('----------------------------------------------------')
    print('Input sentence:', dev_input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    
    print("-Reverse example-")
    target_text = dev_target_texts[seq_index].strip()
    input_seq = dataset.create_encoder_input_item_for_text(target_text)
    input_seq = np.array([input_seq])
#     print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    print('Input sentence:', target_text)
    print('Decoded sentence:', decoded_sentence)




----------------------------------------------------
Input sentence: When 'd you last see him ?
Decoded sentence: When will you be back ?

-Reverse example-
Input sentence: When was the last time you saw him ?
Decoded sentence: When will you be back ?

----------------------------------------------------
Input sentence: Anyone who can verify that ?
Decoded sentence: Is there anything you need ?

-Reverse example-
Input sentence: Can anyone corroborate that ?
Decoded sentence: Can anyone hear me ?

----------------------------------------------------
Input sentence: I 'm not promising anything .
Decoded sentence: I don 't feel so good .

-Reverse example-
Input sentence: No promises , okay ?
Decoded sentence: There 's no need to .

----------------------------------------------------
Input sentence: Nothing 's changed .
Decoded sentence: Nothing that matters .

-Reverse example-
Input sentence: Things ain 't no different .
Decoded sentence: Things will be fine .

-----------------------

### Metric

In [47]:
import rouge

In [48]:
dev_data_path = 'opusparcus_v2/en-dev.txt'
dev_lines = load_data(data_path)

In [49]:
evaluator = rouge.Rouge(['rouge-l'])

In [50]:
def evaluate(input_texts, target_texts, evaluator):
    predicted_texts = []
    for input_text in input_texts:
        input_seq = dataset.create_encoder_input_item_for_text(input_text)
        input_seq = np.array([input_seq])

        decoded_sentence = decode_sequence(input_seq)

        predicted_texts.append(decoded_sentence)

    return evaluator.get_scores(target_texts, predicted_texts, avg=True)

In [53]:
scores = evaluate(dev_input_texts, dev_target_texts, evaluator)

In [54]:
scores

{'rouge-l': {'f': 0.2115507467285711,
  'p': 0.22370623902643574,
  'r': 0.2090855778047892}}